### OpenAIEmbeddings
* 문서의 내용을 수치적인 벡터로 변환하는 과정
* 문서의 의미를 수치화하고 모델들이 문맥적 정보를 이해할 수 있게 한다.
* 벡터화 된 문서들은 문서 분류, 감정 분석, 문서 간 유사도 계산에 사용될 수 있다.

지원되는 모델 목록

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT |
|------------------------|------------------|---------------------------|-----------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      |

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
text = "임베딩 테스트 하기 위한 샘플 문장"

**쿼리 임베딩**
* embedding.embed_query(text)
    * 주어진 텍스트를 임베딩 벡터로 변환하는 함수
    * 텍스트를 벡터공간에 매핑하여 의미적으로 유사한 텍스트를 찾거나 유사도를 계산할 때 사용

In [4]:
query_result = embeddings.embed_query(text)

In [5]:
query_result[:5]

[-0.004322961904108524,
 0.036349765956401825,
 0.030112629756331444,
 -0.019303826615214348,
 0.009104208089411259]

**Document 임베딩**
* embeddings.embed_documents()
    * 함수를 사용하여 텍스트 문서를 임베딩
    * [text]를 인자로 전달하여 단일 문서를 리스트 형태로 임베딩 함수에 전달

In [6]:
doc_result = embeddings.embed_documents(
    [text, text, text, text]
)

In [7]:
len(doc_result)

4

In [8]:
doc_result[0][:5]

[-0.004322961904108524,
 0.036349765956401825,
 0.030112629756331444,
 -0.019303826615214348,
 0.009104208089411259]

### 차원 지정
* text-embedding-3-small 모델 같은 경우 1536차원의 임베딩을 반환

In [9]:
len(doc_result[0])

1536

In [10]:
# 차원이 1024인 text_embedding-3-small 모델 
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [11]:
doc_result_1024 = embeddings_1024.embed_documents([text, text, text, text])
len(doc_result_1024[0])

1024

* 유사도 계산

In [12]:
sentence1 = "안녕하세요. 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다."
sentence3 = "안녕하세요! 반갑습니다."
sentence4 = "Hi. Nice to meet you."
sentence5 = "I like to eat pizza"

* 코사인 유사도
    * -1 ~ 1 사이의 값을 반환
    * 1에 가까울수록 더 유사
    * 0은 관계 없음
    * -1은 반대됨

In [16]:
!pip install -q scikit-learn

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [20]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [23]:
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9211] 안녕하세요. 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다.
[유사도 0.9586] 안녕하세요. 반갑습니다. 	 <=====> 	 안녕하세요! 반갑습니다.
[유사도 0.4726] 안녕하세요. 반갑습니다. 	 <=====> 	 Hi. Nice to meet you.
[유사도 0.0944] 안녕하세요. 반갑습니다. 	 <=====> 	 I like to eat pizza
[유사도 0.9180] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요! 반갑습니다.
[유사도 0.4800] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi. Nice to meet you.
[유사도 0.0890] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat pizza
[유사도 0.4671] 안녕하세요! 반갑습니다. 	 <=====> 	 Hi. Nice to meet you.
[유사도 0.1019] 안녕하세요! 반갑습니다. 	 <=====> 	 I like to eat pizza
[유사도 0.1691] Hi. Nice to meet you. 	 <=====> 	 I like to eat pizza
